In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import namedtuple
import pandas as pd
import wandb

from src.data.utils import add_padding

In [ ]:
given_surname = "surname"
Config = namedtuple("Config", "pref_path in_path out_path cutoff threshold")
config = Config(
    pref_path=f"s3://familysearch-names/processed/tree-preferred-{given_surname}-aggr.csv.gz",
    in_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-train-unfiltered.csv.gz",
    out_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-train-demo.csv.gz",
    cutoff=10000,
    threshold=0.022,
)

In [ ]:
wandb.init(
    project="nama",
    entity="nama",
    name="46_demo_dataset",
    group=given_surname,
    notes="",
    config=config._asdict()
)

In [ ]:
# read dataset
df = pd.read_csv(config.in_path)

In [ ]:
# read the preferred names
pref_df = pd.read_csv(config.pref_path)
print(len(pref_df))

In [ ]:
if given_surname == "surname":
    pref_df = pref_df[~pref_df["name"].str.contains("son$", na=True)].reset_index()
    print(len(pref_df))

In [ ]:
demo_names = set(pref_df.loc[:config.cutoff, "name"].map(add_padding))
print(len(demo_names))
print(next(iter(demo_names)))

In [ ]:
demo_df = df[df["name1"].isin(demo_names) & df["name2"].isin(demo_names)]
print(len(demo_df))
demo_df

In [ ]:
# above threshold
demo_df = demo_df[demo_df["weighted_count"] >= config.threshold]

In [ ]:
print(len(demo_df))
print(len(demo_df["name1"].unique()))
print(len(demo_df["name2"].unique()))

In [ ]:
probe_name = "<richard>"
print(demo_df[(demo_df["name1"] == probe_name) | (demo_df["name2"] == probe_name)])

In [ ]:
demo_df.to_csv(config.out_path, index=False)

In [ ]:
wandb.finish()